In [10]:
import pandas as pd
import numpy as np
import os
import json
import pprint
import matplotlib.pylab as pylab
from pylab import plot,show
import numpy as np
import cPickle as pickle
import time
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import math
from sklearn import cross_validation

%matplotlib inline

# Limit rows disp# Limit rows displayed in notebook
pd.set_option('display.max_rows', 20)
pd.set_option('display.precision', 2)
pylab.rcParams['figure.figsize'] = 16, 12

Loading business details

In [11]:
pd_lasvegas = pickle.load( open('pd_lasvegas.pkl', 'rb'))
'''
pd_lasvegas = pd_lasvegas[np.isfinite(pd_lasvegas[u'attributes_Price Range'])]
pd_lasvegas.rename(columns={'date_x': 'review_date'}, inplace=True)
pd_lasvegas.rename(columns={'date_y': 'start_date'}, inplace=True)
pd_lasvegas.rename(columns={'stars_y': 'stars'}, inplace=True)
pd_lasvegas.reset_index(level=0, inplace=True)
'''
pd_old_business_vegas = pd_lasvegas[pd_lasvegas['new'] == False]
pd_new_business_vegas = pd_lasvegas[pd_lasvegas['new'] == True]

Creating business vector

In [17]:
''' Business Attributes '''
attrs = [u'attributes_Accepts Credit Cards', 
         u'attributes_Accepts Insurance',
         u'attributes_Ages Allowed', 
         u'attributes_Alcohol',
         u'attributes_Ambience', 
         u'attributes_Attire', 
         u'attributes_BYOB',
         u'attributes_BYOB/Corkage', 
         u'attributes_By Appointment Only',
         u'attributes_Caters', 
         u'attributes_Coat Check',
         u'attributes_Corkage', 
         u'attributes_Delivery',
         u'attributes_Dietary Restrictions', 
         u'attributes_Dogs Allowed',
         u'attributes_Drive-Thru', 
         u'attributes_Good For Dancing', 
         u'attributes_Good For Groups',
         u'attributes_Good For Kids', 
         u'attributes_Hair Types Specialized In', 
         u'attributes_Happy Hour',
         u'attributes_Has TV', 
         u'attributes_Music',
         u'attributes_Noise Level', 
         u'attributes_Open 24 Hours',
         u'attributes_Order at Counter', 
         u'attributes_Outdoor Seating',
         u'attributes_Parking', 
         u'attributes_Payment Types',
         u'attributes_Price Range', 
         u'attributes_Smoking',
         u'attributes_Take-out', 
         u'attributes_Takes Reservations',
         u'attributes_Waiter Service', 
         u'attributes_Wheelchair Accessible',
         u'attributes_Wi-Fi', 
         u'categories',
         u'hours_Friday', 
         u'hours_Monday', 
         u'hours_Saturday',
         u'hours_Sunday', 
         u'hours_Thursday', 
         u'hours_Tuesday',
         u'hours_Wednesday',
         'distance'
        ]

def find_distance(o_bid, n_bid, old, new):
    lat1=old['latitude']
    long1=old['longitude']
    lat2=new['latitude']
    long2=new['longitude']
    if lat1==lat2 and long1==long2:
        return 0
    # Convert latitude and longitude to 
    # spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0
         
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
         
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
          
    # Compute spherical distance from spherical coordinates.
    try:     
        cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) + 
               math.cos(phi1)*math.cos(phi2))
        arc = math.acos( cos )
    except: 
        print str(lat1)+","+str(long1)+","+str(lat2)+","+str(long2)
        print o_bid
        print n_bid
    
    #multiply with radius of earth in kms to get the actual value
    return arc*6371

def getHourAttr(old, new):
    if old[u'close'] == new[u'close'] and old[u'open'] == new[u'open']:
        return 1
    else:
        return -1

def getParkingAttr(old, new):
    parking_keys = [u'garage', u'street', u'validated', u'lot', u'valet']
    '''if np.isnan(old['attributes_Parking']) or math.isnan(new['attributes_Parking']):
        return -1
    '''
    for key in parking_keys:
        try:
            if old['attributes_Parking'][key]!= new['attributes_Parking'][key]:
                return -1
        except KeyError, e:
            '''print 'I got a KeyError - reason "%s"' % str(e)
            print "Old: ", old
            print "New: ", new
            break'''
            return -1
    return 1

def getCategoryAttr(old, new):
    #print old, new
    oldCategories = set(old.encode('utf8').split(','))
    newCategories = set(new.encode('utf8').split(','))
    L = len(oldCategories.intersection(newCategories))
    return L
    
def getVector(old, new):
    res = []
    for attr in attrs:
        if attr == 'distance':
            res.append(find_distance(0, 0, old, new))
            continue
            
        if pd.isnull(old[attr]) and pd.isnull(new[attr]):
            res.append(1)
            continue
            
        elif pd.isnull(old[attr]) or pd.isnull(new[attr]):
            res.append(-1)
            continue

        if attr == u'attributes_Price Range':
            if new[attr].astype(float) < old[attr].astype(float):
                res.append(-10)
            else:
                res.append(0)
            continue
            
        if 'hours_' in attr:
            res.append(getHourAttr(old[attr], new[attr]))
            continue
            
        if attr == u'categories':
            res.append(getCategoryAttr(old[attr], new[attr]))
            continue
            
        if attr == u'attributes_Parking':
            res.append(getParkingAttr(old, new))
            continue

        elif old[attr] == new[attr]:
            res.append(1)
        else:
            res.append(-1)
    return res

#Testing Out Vector Nonsense 
c = 0
pd_business =  pd_lasvegas[pd_lasvegas['cluster'] == c]
pd_old_business = pd_business[pd_business['new'] == False]
pd_new_business = pd_business[pd_business['new'] == True]
print getVector(pd_old_business.iloc[0], pd_new_business.iloc[0])

[1, 1, 1, -1, -1, -1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 0, 1, -1, -1, -1, -1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 4.9970971697914965]


Calculating Conditional Mean For Each Business

In [18]:
import datetime
''' Helper for Date in Pandas'''
def todate(d):
    return datetime.datetime.strptime(d, '%Y-%m-%d')


''' Calculate mean_after - mean_before : returns +1 if mean improves -1 if mean reduces'''
def calcAverage(df):
    #print "Printing DF", df
    df_before = df[df['before'] == True]
    df_after = df[df['before'] == False]
    
    #print "before", df_before
    #print "after", df_after
    
    l1 = len(df_before['stars'])
    l2 = len(df_after['stars'])

    if l1 == 0 or l2 == 0:
        return -1
    else:
        a = sum(df_before['stars']) / (len(df_before['stars'])*1.0)
        b = sum(df_after['stars']) / (len(df_after['stars'])*1.0)
        if a - b > 0:
            return -1
        else:
            return 1
        
def createTrainingData(c=0, date_range=60, sampleAll=True, maxsamples=0):
    # Use cluster 0,1 for training
    X, Y = [], []

    diff = datetime.timedelta(days=date_range)
    pd_business_cluster =  pd_lasvegas[pd_lasvegas['cluster'] == c]
    pd_old_business = pd_business_cluster[pd_business_cluster['new'] == False]
    #pd_old_business_Y = pd_business_cluster[["business_id", "review_date", "stars"]] # This is to simplify Y calculation
    pd_new_business = pd_business_cluster[pd_business_cluster['new'] == True]
    gb_new_business = pd_new_business.groupby('business_id')
    gb_old_business = pd_old_business.groupby('business_id')

    
    print "Calculating vector for cluster ", c
    start_time = time.time()
    for new_business_id, new_business_details in gb_new_business:
        start_date = todate(pd_new_business.iloc[0]['start_date'])
        new_business = new_business_details.iloc[0]
        
        ''' Calculating Y for Current New Business '''
        # Getting all old business reviews between start_date -diff to start_date + diff
        pd_old_business_tempY = pd_old_business[pd_old_business.review_date.apply(todate) <= start_date + diff]
        pd_old_business_tempY = pd_old_business_tempY[pd_old_business_tempY.review_date.apply(todate) >= start_date - diff]
        
        # Labelling All Reviews Before Start Date as True
        pd_old_business_tempY['before'] = (pd_old_business_tempY['review_date'].apply(todate) < start_date)
        
        #print pd_old_business_tempY.groupby('business_id')
        gb_old_business_tempY = pd_old_business_tempY.groupby(['business_id'])
        temp_y = gb_old_business_tempY.apply(calcAverage).values.tolist()
        
        ''' Calculating X for Current New Business '''
        temp_x = []
        for old_business_id, old_business_details in gb_old_business_tempY:
            old_business = old_business_details.iloc[0]
            #print old_business
            temp_x.append(getVector(old_business, new_business))
            
            
        X.extend(temp_x)
        Y.extend(temp_y)
        
        if sampleAll == False and len(X) > maxsamples:
            break

    end_time = time.time()
    print "Time taken for cluster ", c, " is ", end_time - start_time
    print "Completed calculating vector for cluster ", c
    print len(X)
    print len(Y)
    return X, Y

for i in [60]:
    print "Data for : ", i
    X, Y = createTrainingData(c=0, date_range=i)
    #pickle.dump(X, open('X_'+str(i)+'.pkl', 'wb'))
    #pickle.dump(Y, open('Y_'+str(i)+'.pkl', 'wb'))

Data for :  60
Calculating vector for cluster  0
Time taken for cluster  0  is  584.216160059
Completed calculating vector for cluster  0
66123
66123


Learning Algorithm: Logistic Regression/SVM

In [20]:
from sklearn import neighbors, linear_model

for i in [60]:
    print "At cluster ", i
    #X = pickle.load(open('X_'+str(i)+'.pkl', 'rb'))
    #Y = pickle.load(open('Y_'+str(i)+'.pkl', 'rb'))
    X_np, Y_np = np.array(X), np.array(Y)
    logistic = linear_model.LogisticRegression()
    scores = cross_validation.cross_val_score(logistic, X_np, Y_np, cv=5)
    print "Regression Scores: ", scores

#sampling_mask_train =np.random.randint(len(X),size=10000)
#sampling_mask_test =np.random.randint(len(X),size=2500)
#X_train, Y_train = X_np[sampling_mask_train,:], Y_np[sampling_mask_train]
#X_test, Y_test = X_np[sampling_mask_test,:], Y_np[sampling_mask_test]

#clf = SVC(kernel='rbf')
#print('SVM score: %f' % clf.fit(X_train, Y_train).score(X_test, Y_test))
#scores = cross_validation.cross_val_score(clf, X_np, Y_np, cv=5)
#print  "SVM Scores: " , scores

#clf2 = SVC(kernel='linear')
#print('Linear SVM score: %f' % clf2.fit(X_train, Y_train).score(X_test, Y_test))
#clf3 = SVC(kernel='poly')
#print('Poly Linear SVM score: %f' % clf3.fit(X_train, Y_train).score(X_test, Y_test))
#clf4 = SVC(kernel='sigmoid')
#print('Sigmoid Linear SVM score: %f' % clf4.fit(X_train, Y_train).score(X_test, Y_test))
#clf = SVC(kernel='rbf')
#print('SVM score: %f' % clf.fit(X_train, Y_train).score(X_test, Y_test))

At cluster  60
Regression Scores:  [ 0.66633903  0.66381853  0.66643981  0.66250756  0.65819722]
